## Code Details
Author: Rory Angus<br>
Created: 21FEB19<br>
Version: 0.1<br>
***
This code is to test a writing of data to a MongoDB. <br>
This is a proof of concept and the data is real. However, it does not bring all of it into Mongo, only the key fields.
This uses data that was extracted after the SSO data model was implemented at LE on the platform.
The data now is in three parts. The groups and their members, the users linked to the results as well as coaching/coachee relationship.
Please note that the results from doing the survey can be more than two per journey and the coaching/coachee relationship is many to many. There is a field called coach on the user but that is from the old version and should not be used.<br>

There is also an issue with the data migration onto the new platform which ended up with some users having coaching relationships with UTS students but they are not part of the UTS organisation on the platform. This is no longer possible, so when it happens, this will be patched manually in this code.

This code is to load the list of users onto Mongo to help with the looking up of the coaches name.

### I needed to manually add the missing the coaches to this dataset.

# Package Importing + Variable Setting

In [1]:
import pandas as pd
import numpy as np

import datetime

# mongo stuff
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import bson

# json stuff
import json

In [2]:
# the file to read. This needs to be manually updated
readLoc = "~/datasets/CLARA/190328_052400_LE_LivePlatform_ListOfUsers.json"
readLocPam = "~/datasets/CLARA/190328_052400_LE_LivePlatform_ListOfUsersPAM.json"
# if true the code outputs to the notebook a whole of diagnostic data that is helpful when writing but not so much when running it for real
verbose = False
# first run will truncate the target database and reload it from scratch. Once delta updates have been implmented this needs adjusting
first_run = True

# Set display options

In [3]:
# further details found by running:
# pd.describe_option('display')
# set the values to show all of the columns etc.
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

# locals() # show all of the local environments

# Connect to Mongo DB

In [4]:
# create the connection to MongoDB
# define the location of the Mongo DB Server
# in this instance it is a local copy running on the dev machine. This is configurable at this point.
client = MongoClient('127.0.0.1', 27017)

# define what the database is called.
db = client.CLARA

# define the collection
raw_data_collection = db.raw_data_claraUsers

Command to clean the databzse if needed when running this code

In [5]:
# Delete the raw_data_collection - used for testing
if first_run:
    raw_data_collection.drop()

# Functions Definitions

In [6]:
# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

# Place Data from JSON File into Mongo

In [7]:
#import the data file
claraDf = pd.read_json(readLoc, orient='records')

In [8]:
# read Pams record as it exists in a different organisation and needs to be merged
pamDf = pd.read_json(readLocPam, orient='records')

In [9]:
# add Pam's record onto the end of the other ones stored in claraDf
claraDf = pd.concat([claraDf, pamDf], ignore_index=True)

In [10]:
if verbose:

    # count columns and rows
    print("Number of columns are " + str(len(claraDf.columns)))
    print("Number of rows are " + str(len(claraDf.index)))
    print()

    # output the shape of the dataframe
    print("The shape of the data frame is " + str(claraDf.shape))
    print()

    # output the column names
    print("The column names of the data frame are: ")
    print(*claraDf, sep='\n')
    print()

    # output the column names and datatypes
    print("The datatypes of the data frame are: ")
    print(claraDf.dtypes)
    print()

In [11]:
# mongo is not able to store integers so convert them to strings

claraDf['orgUser_Id'] = claraDf['orgUser_Id'].astype(str)
claraDf['isSSO'] = claraDf['isSSO'].astype(str)

# output the column names and datatypes
if verbose:
    print("The datatypes of the data frame are: ")
    print(claraDf.dtypes)
    print()

In [12]:
# Loop through the data frame and build a list
# the list will be used for a bulk update of MongoDB

# define the list to hold the data
clara_row = []

# loop through dataframe and create each item in the list
for index, row in claraDf.iterrows():
    clara_row.insert(
        index, {
            "user_index":
            index,
            "displayName":
            claraDf['displayName'].iloc[index],
            "clientUserId":
            claraDf['clientUserId'].iloc[index],
            "nameId":
            claraDf['nameId'].iloc[index],
            "primaryEmail":
            claraDf['primaryEmail'].iloc[index],
            "declaraLinked":
            claraDf['declaraLinked'].iloc[index],
            "languagePreference":
            claraDf['languagePreference'].iloc[index],
            "AvatarSupplied":
            claraDf['avatarSupplied'].iloc[index],
            "userStatus":
            claraDf['status'].iloc[index],
            "additionalData":
            claraDf['additionalData'].iloc[index],
            "userDeletedAt":
            claraDf['deletedAt'].iloc[index],
            "coachId":
            claraDf['coachId'].iloc[index],
            "orgUser_Id":
            claraDf['orgUser_Id'].iloc[index],
            "orgUserId":
            claraDf['orgUserId'].iloc[index],
            "learningPlatformUserId":
            claraDf['learningPlatformUserId'].iloc[index],
            "isSSO":
            claraDf['isSSO'].iloc[index],
            "insertdate":
            datetime.datetime.utcnow()
        })

if verbose:
    print(clara_row[6:8])

In [13]:
# bulk update the mongo database

raw_data_collection.insert_many(clara_row)

if verbose:
    print(raw_data_collection.inserted_ids)

## Create Index

In [14]:
# Only create the indexes onthe first run through
if first_run:
    # put the restult into a list so it can be looked at later.
    result = []

    # Create some indexes
    result.append(
        raw_data_collection.create_index([('user_index', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('clientUserId', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('displayName', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('orgUser_Id', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('dateFrom', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index(
            [('userClientUserId', pymongo.ASCENDING)], unique=False))
    result.append(
        raw_data_collection.create_index([('userCoachId', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('userStatus', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('nameId', pymongo.ASCENDING)],
                                         unique=False))
    result.append(
        raw_data_collection.create_index([('primaryEmail', pymongo.ASCENDING)],
                                         unique=False))

    if verbose:
        print(result)